# Import Libraries and dataset to use (Seperate the dataset to 100 cats and 100 dogs images only)

In [11]:
import os
import shutil
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

In [5]:
source_dir = r"C:/Users/Miggy/Documents/Applied Data Science with Python/Activity 4/archive/Animals"
base_dir = "dataset_100"


for label in ['cats', 'dogs']:
    os.makedirs(os.path.join(base_dir, 'train', label), exist_ok=True)


for label in ['cats', 'dogs']:
    src_folder = os.path.join(source_dir, label)
    dst_folder = os.path.join(base_dir, 'train', label)
    images = os.listdir(src_folder)[:100]
    for img in images:
        shutil.copy(os.path.join(src_folder, img), os.path.join(dst_folder, img))



### Apply appropriate preprocessing techniques, including resizing, normalization, and data augmentation if needed.

In [6]:
image_size = (224, 224)  
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


### Select two pre-trained convolutional neural network models (e.g., VGG16, ResNet50, MobileNetV2) to use as the base for your transfer learning approach.

In [8]:
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


### Use the selected pre-trained models to extract features from your dataset. Train a classifier (e.g., dense layers) on top of the extracted features. (10 - 15 epochs)

In [9]:
def train_model(base, name):
    for layer in base.layers:
        layer.trainable = False

    model = build_model(base)
    model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=15
    )
    return model, history


In [10]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

vgg_model, vgg_history = train_model(vgg_base, "VGG16")
mobilenet_model, mobilenet_history = train_model(mobilenet_base, "MobileNetV2")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


c:\Users\Miggy\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5148 - loss: 0.7380 - val_accuracy: 0.6250 - val_loss: 0.6624
Epoch 2/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5143 - loss: 0.6860 - val_accuracy: 0.8000 - val_loss: 0.6309
Epoch 3/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.6010 - loss: 0.6601 - val_accuracy: 0.6750 - val_loss: 0.6247
Epoch 4/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.6614 - loss: 0.6132 - val_accuracy: 0.7750 - val_loss: 0.6096
Epoch 5/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.6863 - loss: 0.6106 - val_accuracy: 0.8000 - val_loss: 0.5882
Epoch 6/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7554 - loss: 0.5642 - val_accuracy: 0.7750 - val_loss: 0.5631
Epoch 7/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.8014 - loss: 0.5428 - val_accuracy: 0.8750 - val_loss: 0.5455
Epoch 8/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7661 - loss: 0.5636 - val_accuracy: 0.8250 - val_loss: 0.5469
Epoch 9/15
5/5 

### Compare the performance of the three models based on accuracy and other relevant metrics.

In [ ]:
def evaluate_model(model, generator):
    generator.reset()
    preds = model.predict(generator, verbose=0)
    y_pred = (preds > 0.5).astype(int)
    y_true = generator.classes

    print(classification_report(y_true, y_pred, target_names=generator.class_indices.keys()))
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:\n", cm)

evaluate_model(vgg_model, val_generator)
evaluate_model(mobilenet_model, val_generator)